In [1]:
import requests
import tensorflow as tf
import base64
import nltk
import re
import os
from dotenv import load_dotenv
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from pprint import PrettyPrinter
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acqmallatief/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/acqmallatief/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
stemmer = PorterStemmer()
stop_words = stopwords.words('english')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\.\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('\w*\d\w*', '', text)
    tokens = word_tokenize(text)
    cleaned_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'text': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

In [7]:
end_point = os.getenv("API_PREDICTION")

def predict(input):
    text = clean_text(input)
    text = text.encode('utf-8')
    example = serialize_text(text)
    json_data = {
        "signature_name":"serving_default",
        "instances":[
            {
                "examples":{"b64": base64.b64encode(example).decode('utf-8')}
            }
        ]
    }
    response = requests.post(end_point, json=json_data)
    prediction = tf.argmax(response.json()["predictions"][0]).numpy()
    map_labels = {0: "sadness",
                  1: "joy",
                  2: "love",
                  3: "anger",
                  4: "fear",
                  5: "surprise"}
    return map_labels[prediction]

In [8]:
text = input("masukkan kalimat:")

print(f"{text}\n classifisi:{predict(text)}")

i am happy
 classifisi:joy
